In [1]:
from dotenv import dotenv_values
from openai import OpenAI
import instructor
from pydantic import BaseModel
from typing import Optional
import pandas as pd
from pycaret.regression import load_model, predict_model

In [2]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

In [3]:
instructor_openai_client = instructor.from_openai(openai_client)

## Zamykamy model w funkcji

In [4]:
 # Konwersja tekstu do formatu json
def retrieve_structure(text: str, response_model: BaseModel):
    response = instructor_openai_client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        response_model=response_model,
        messages=[
            {
                "role": "user",
                "content": text,
            },
        ],
    )
    
    return response.model_dump()

In [5]:
def validate_response_data(person):
    if (person.gender in [None, ''] or 
        person.age is None or 
        person.weight is None or 
        person.height is None or 
        person.children is None or
        person.smoker in [None, ''] or 
        (person.region.lower() != 'północny wschód' and person.region.lower() != 'południowy wschód' and person.region.lower() != 'północny zachód' and person.region.lower() != 'południowy zachód')): 
        return None
    else:
        # Upewnij się, że wartości są poprawne przed porównaniem
        if (person.age > 0 and person.weight > 0 and person.height > 0 and person.children >= 0):  # Przykładowe porównania
            return person
        else:
            print("🛑 Niepoprawne wartości dla jednego z pól!")
            return None

In [6]:
class SpeechPerson(BaseModel):
    gender: Optional[str]
    age: Optional[int]
    weight: Optional[int]
    height: Optional[float]
    children: Optional[int]
    smoker: Optional[str]
    region: Optional[str]

In [12]:
result_data = retrieve_structure(
    text="Mam 40 lat, jestem Kobietą, ważę 75kg, mam 195 wzrostu, mam 1 dziecko, palę, mieszkam w północnym wschodzie",
    response_model=SpeechPerson
    )

                # Create an instance of response_model to leverage Pydantic validation
person = SpeechPerson(**result_data)
validated_person = validate_response_data(person=person)

if validated_person is None:
    print("🛑 Brak jakiejś informacji! Nagraj notatkę jeszcze raz!")
else:
    if validated_person.height > 3.0:
        speech_bmi = validated_person.weight / ((validated_person.height / 100) ** 2)
        print(speech_bmi)
    else:
        speech_bmi = validated_person.weight / (validated_person.height ** 2)
        print(speech_bmi)

19.723865877712033


In [13]:
speech_person_df = pd.DataFrame([{
    "age": validated_person.age,
    "sex": validated_person.gender.lower(),
    "bmi": round(speech_bmi, 2),
    "children": validated_person.children,
    "smoker": validated_person.smoker.lower(),
    "region": validated_person.region.lower(),
}])
speech_person_df

,age,sex,bmi,children,smoker,region
0,40,kobieta,19.72,1,tak,północny wschód


In [14]:
def get_model():
    return load_model('v4_insurance_charge_regression')

model = get_model()

def predict_cost(model, data):
    prediction = predict_model(model, data=data)
    return round(prediction["prediction_label"][0], 2)

Transformation Pipeline and Model Successfully Loaded


In [15]:
predicted_charge = predict_cost(model, speech_person_df)
predicted_charge

17786.19

In [16]:
if speech_bmi > 24.9:
    new_weight = person.weight - 10
    new_bmi = new_weight / (person.height ** 2)
    speech_person_df_lower_bmi = speech_person_df.copy()
    speech_person_df_lower_bmi["bmi"] = round(new_bmi, 2)
    cost_lower_bmi = predict_cost(model, speech_person_df_lower_bmi)
    if cost_lower_bmi < predicted_charge:
        if person.smoker == "tak":
            print(f"- Zredukuj masę o 10 kg, a Twój koszt ubezpieczenia wyniesie {cost_lower_bmi}!")
        else:
            print(f"- Zredukuj masę o 10 kg, a Twój koszt ubezpieczenia wyniesie {cost_lower_bmi}")

if speech_bmi < 18.5:
    new_weight = person.weight + 10
    new_bmi = new_weight / (person.height ** 2)
    speech_person_df_higher_bmi = speech_person_df.copy()
    speech_person_df_higher_bmi["bmi"] = round(new_bmi, 2)
    cost_higher_bmi = predict_cost(model, speech_person_df_higher_bmi)
    if cost_higher_bmi < predicted_charge:
        if person.smoker == "tak":
            print(f"- Zwiększ masę o 10 kg, a Twój koszt ubezpieczenia wyniesie {cost_higher_bmi}")
        else:
            print(f"- Zwiększ masę o 10 kg, a Twój koszt ubezpieczenia wyniesie {cost_higher_bmi}")